In [1]:
# your parameters here...
upstream = ['fetch_n_filter_gdelt_bq']

In [2]:
# Parameters
upstream = {
    "fetch_n_filter_gdelt_bq": {
        "nb": "/Users/aiujdm2/market_watch/output/notebooks/fetch_n_filter_gdelt_bq.ipynb",
        "data": "/Users/aiujdm2/market_watch/output/data/raw/gdelt-gkg-bq.csv",
    }
}
product = {
    "nb": "/Users/aiujdm2/market_watch/output/notebooks/clean_gdelt_data.ipynb",
    "data": "/Users/aiujdm2/market_watch/output/data/interim/cleaned-gdelt-data.csv",
}


In [3]:
# your code here...
import pandas as pd
import numpy as np
from pathlib import Path

In [4]:
input_file_path = upstream['fetch_n_filter_gdelt_bq']['data']

data_df = pd.read_csv(input_file_path, index_col=0)
print(data_df.columns)

Index(['GKGRECORDID', 'DATE', 'SourceCollectionIdentifier',
       'DocumentIdentifier', 'Locations', 'Persons', 'Organizations', 'Tone'],
      dtype='object')


In [5]:
def split_locations(location_list):
    location_names = []
    if type(location_list) is not float:
        for location_string in location_list:
            loc_parts = location_string.split('#')
            location_names.append(loc_parts[1]) if len(loc_parts) > 1 else np.nan
            
    return location_names

data_df.Locations = data_df.Locations.str.split(';').apply(split_locations)

In [6]:
#  Clean some data elements
data_df.Persons = data_df.Persons.str.findall(pat="[A-Z][a-z]+ [A-Z][a-z]+")

In [7]:
data_df.Organizations = data_df.Organizations.str.replace("Tesla|Tesla Motors|Tesla Inc", '').str.findall(pat="[A-Z][a-z]+ [A-Z][a-z]+")

/var/folders/b0/4tc1nfbd6z7_vhx4nxcj2n91c0l9h6/T/ipykernel_18971/2748311790.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  data_df.Organizations = data_df.Organizations.str.replace("Tesla|Tesla Motors|Tesla Inc", '').str.findall(pat="[A-Z][a-z]+ [A-Z][a-z]+")


In [8]:
data_df.Tone = data_df.Tone.str.split(',')

In [9]:
# Clean Tone
data_df['AvgTone'] = data_df.Tone.apply(lambda x: x[0])
data_df['PosScore'] = data_df.Tone.apply(lambda x: x[1])
data_df['NegScore'] = data_df.Tone.apply(lambda x: x[2])
data_df['Polarity'] = data_df.Tone.apply(lambda x: x[3])

In [10]:
data_df.drop(["Tone", "DATE", "SourceCollectionIdentifier", "DocumentIdentifier"], axis = 1, inplace=True)

In [11]:
output_file_path = product['data']
Path(output_file_path).parent.mkdir(exist_ok=True, parents=True)
data_df.to_csv(output_file_path)
print(f"Saved file {output_file_path}")

Saved file /Users/aiujdm2/market_watch/output/data/interim/cleaned-gdelt-data.csv
